In [29]:

import time

In [49]:
import requests
import json
# import pandas as pd
# from datetime import datetime

# API Configuration
BASE_URL = "http://localhost:8001"
HEADERS = {"Content-Type": "application/json"}

def post_request(endpoint: str, payload: dict, headers: dict, timeout: int = 15):
    url = f"{BASE_URL}/{endpoint}"
    
    try:
        response = requests.post(
            url,
            json=payload,
            headers=headers,
            timeout=timeout
        )
        
        print(f"\n📡 POST to {endpoint}")
        print(f"Status Code: {response.status_code}")
        print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Request successful!")
            return result
        else:
            print(f"❌ Request failed: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return None

## Product matching (With Image)

In [57]:
def product_matching(caption_payload, user_messages):
    start = time.time()

    ## Image Captioning request
    start = time.time()
    if caption_payload["image_path"] == "":
        print("No image provided for captioning.")
        image_caption = "No image provided for captioning."
    else:
        print("image captioning...")
        result = post_request("image_captioning", caption_payload, HEADERS)
        image_caption = result['results']

    print("find docs...")
    doc_payload = {
        "user_query": f'''{user_messages}''',
        "image_prompt": image_caption
    }

    result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
    reference = result['results']
    print(reference)


    print("generate response...")
    payload = {
        "question": f'''{user_messages}''',
        "reference": f'''{reference}'''
    }

    result = post_request("generation", payload, HEADERS, timeout=30)
    print(result["response"])

    end = time.time()
    print(f"Total time taken: {end - start:.2f} seconds")
    return result["response"], reference

In [62]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

product_matching(caption_payload, user_messages)

image captioning...

📡 POST to image_captioning
Status Code: 200
Response Time: 3.86s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 3.94s
✅ Request successful!
[{'search_term': 'floral maxi dress', 'search_results': [{'score': 0.7890574, 'id': 'Image40.jpg', 'name': 'Yellow Floral Spaghetti Strap Midi Dress', 'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe: Romantic Summer / Whimsical Chic. Fit: Fitted bust with a flowing A-line midi skirt. Material: Lightweight, airy fabric (likely chiffon or georgette) with a subtle sheen. Care: Dry clean recommended or delicate hand wash. Matches Well With: Outerwear: A light white cardigan

('Based on the product information, here are some recommendations:\n\nFor the beach party:\n1. Yellow Floral Spaghetti Strap Midi Dress ($98.00) - Romantic summer style\n2. Navy V-Neck Tiered Maxi Dress ($85.00) - Resort chic look\n\nFor the formal dinner:\n1. Dusty Rose Pleated Midi Dress ($95.00) - Vintage feminine elegance\n2. Yellow Floral Spaghetti Strap Midi Dress ($98.00) - Can be dressed up for a formal occasion\n\nWould you like more details on any of these options?',
 [{'search_term': 'floral maxi dress',
   'search_results': [{'score': 0.7890574,
     'id': 'Image40.jpg',
     'name': 'Yellow Floral Spaghetti Strap Midi Dress',
     'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical 

In [64]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }
product_matching(caption_payload, user_messages)

No image provided for captioning.
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 5.51s
✅ Request successful!
[{'search_term': 'floral maxi dress', 'search_results': [{'score': 0.7890574, 'id': 'Image40.jpg', 'name': 'Yellow Floral Spaghetti Strap Midi Dress', 'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe: Romantic Summer / Whimsical Chic. Fit: Fitted bust with a flowing A-line midi skirt. Material: Lightweight, airy fabric (likely chiffon or georgette) with a subtle sheen. Care: Dry clean recommended or delicate hand wash. Matches Well With: Outerwear: A light white cardigan or a cropped denim jacket. Shoes: Strappy heeled sandals, elegant mules, 

("For a beach party, I recommend exploring some lovely floral maxi dresses or off-shoulder sundresses that will keep you looking cute and feeling comfortable. They're perfect for a relaxed summer event. For your formal dinner, our elegant belted midi dresses or sophisticated satin-style options would be wonderful choices. I suggest browsing through our search results to find the exact piece that catches your eye. Would you like me to describe some of the styles in more detail?",
 [{'search_term': 'floral maxi dress',
   'search_results': [{'score': 0.7890574,
     'id': 'Image40.jpg',
     'name': 'Yellow Floral Spaghetti Strap Midi Dress',
     'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrica

### Product Matching (Without Image)

In [ ]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

## Style matching

In [67]:
def style_matching(caption_payload, user_messages):
    start = time.time()
    if caption_payload["image_path"] == "":
        print("No image provided for captioning.")
        image_caption = "No image provided for captioning."
    else:
        print("image captioning...")
        result = post_request("image_captioning", caption_payload, HEADERS)
        image_caption = result['results']

    print("style complement...")
    doc_payload = {
        "user_query": f'''{user_messages}''',
        "image_prompt": image_caption
    }
    result = post_request("style_complement", doc_payload, HEADERS, timeout=30)
    style_complement = result['results']
    print(style_complement)
    print("find docs...")
    doc_payload = {
        "user_query": style_complement,
        "image_prompt": image_caption
    }

    result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
    reference = result['results']
    print(reference)


    print("generate response...")
    payload = {
        "question": f'''{user_messages}''',
        "reference": f'''{reference}'''
    }

    result = post_request("generation", payload, HEADERS, timeout=30)
    print(result["response"])

    end = time.time()
    print(f"Total time taken: {end - start:.2f} seconds")
    return result["response"], reference
        

In [54]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

In [55]:
style_matching(caption_payload, user_messages)

image captioning...

📡 POST to image_captioning
Status Code: 200
Response Time: 2.77s
✅ Request successful!
style complement...

📡 POST to style_complement
Status Code: 200
Response Time: 4.08s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 6.20s
✅ Request successful!
[{'search_term': 'Belted shirt dress in summer colors', 'search_results': [{'score': 0.7064234, 'id': 'Image12.png', 'name': 'Charcoal Grey Belted Shirt Dress', 'description': 'This sophisticated charcoal grey shirt dress features a structured collar and full-length button placket, exuding a polished and modern aesthetic. Shoulder epaulets add a subtle utilitarian detail, while the matching adjustable belt cinches the waist to create a flattering silhouette. The long sleeves can be rolled up for a more relaxed look. Style Vibe: Office Chic / Modern Professional. Fit: Tailored, with a cinched waist that flares slightly into an A-line skirt. Material: Smooth, mid-weight polye

('Based on the provided product information, here are some summer dress options:\n\nFor a beach party (casual and cute):\n1. Yellow Floral Spaghetti Strap Midi Dress ($98.00)\n- Vibrant floral print in yellow with pink, purple, and orange\n- Flowing A-line silhouette\n- Spaghetti straps and sweetheart neckline\n- Style Vibe: Romantic Summer / Whimsical Chic\n- Matches well with: Cropped denim jacket, strappy heeled sandals\n\nFor a formal dinner:\n1. Charcoal Grey Belted Shirt Dress ($78.00)\n- Sophisticated shirt dress with structured collar\n- Full-length button placket\n- Adjustable belt to cinch the waist\n- Style Vibe: Office Chic / Modern Professional\n- Matches well with: Structured blazer, pointed-toe ankle boots, minimalist clutch\n\n2. Dusty Rose Pleated Midi Dress ($95.00)\n- Elegant midi dress with classic collared neckline\n- Fitted bodice with belt detail\n- Pleated A-line skirt\n- Style Vibe: Vintage Feminine / Classic Elegance\n- Matches well with: Tailored trench coat,

In [56]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

style_matching(caption_payload, user_messages)

No image provided for captioning.
style complement...

📡 POST to style_complement
Status Code: 200
Response Time: 2.83s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 5.10s
✅ Request successful!
[{'search_term': 'casual cute beach party dress', 'search_results': [{'score': 0.69277227, 'id': 'Image29.png', 'name': 'Mustard Yellow Swing Mini Dress', 'description': 'This cheerful mustard yellow mini dress features a simple crew neck and short sleeves, with a comfortable and flattering swing silhouette that flares gently from the bodice. Its vibrant color and effortless design make it a perfect casual piece for bright, sunny days, offering comfort without compromising on style. Style Vibe: Casual & Bright / Playful Everyday. Fit: Relaxed, A-line swing fit. Material: Soft, stretchy jersey knit (cotton blend). Care: Machine wash cold. Matches Well With: Outerwear: A light denim jacket or a white casual cardigan. Shoes: White sneakers, flat san

("Based on the product information provided, here are some recommendations for your summer outfits:\n\nFor the Beach Party:\n1. Mustard Yellow Swing Mini Dress ($45.00) - A casual and bright dress with a comfortable swing silhouette. It's perfect for a beach party and can be paired with white sneakers or flat sandals.\n\n2. Blue Tropical Print Sarong Skirt ($45.00) - A vibrant sarong skirt with a beachy vibe that can be styled with a white bikini top or a lightweight camisole.\n\nFor the Formal Dinner:\n1. Dusty Rose Pleated Midi Dress ($95.00) - An elegant dress with a collared neckline, fitted bodice, and a subtle belt detail. It's perfect for a formal dinner and can be paired with pointed-toe heels or ballet flats.\n\n2. Yellow Floral Spaghetti Strap Midi Dress ($98.00) - A romantic midi dress with a sweetheart neckline and floral print, suitable for a semi-formal dinner. It can be accessorized with strappy heeled sandals and a delicate clutch.\n\nWould you like more details about t

### testing

In [73]:
## Setting input
caption_payload = {
    "image_path": "uploads/09161e93-5171-4dd2-8e27-c0439171c5d1-testsuit.jpg"
}


user_messages = {
        "messages": [
            {"role": "user", "content": "I want a pant that could match with this upper part."},
        ]
    }

style_matching(caption_payload, user_messages)

image captioning...

📡 POST to image_captioning
Status Code: 200
Response Time: 3.62s
✅ Request successful!
style complement...

📡 POST to style_complement
Status Code: 200
Response Time: 2.32s
✅ Request successful!
[
    "charcoal wool dress pants",
    "light gray slim fit trousers",
    "navy blue suit pants",
    "windowpane check suit pants",
    "tapered wool blend dress pants",
    "pinstripe suit trousers",
    "textured gray suit pants"
]
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 4.30s
✅ Request successful!
[{'search_term': 'charcoal wool dress pants', 'search_results': [{'score': 0.6966369, 'id': 'Image75.jpg', 'name': "Men's Black Cargo Pants with Large Pockets", 'description': "These men's black cargo pants are designed with a relaxed fit and feature conspicuously large, functional cargo pockets on the sides, emphasizing a utilitarian and practical aesthetic. The comfortable design, likely with a drawstring or elasticated waist, makes them a 

("I apologize, but I noticed that no specific upper part was mentioned in your request. Without knowing the details of the top you're looking to pair pants with, I can offer some general advice. Based on the search results, I see several versatile trouser styles that could work well in different settings - from navy blue pleated trousers perfect for smart-casual looks to more relaxed cargo pants for a streetwear vibe. I recommend browsing through the search results to find a pair that matches your personal style and the specific top you have in mind. Would you like to share more details about the upper part you're trying to coordinate?",
 [{'search_term': 'charcoal wool dress pants',
   'search_results': [{'score': 0.6966369,
     'id': 'Image75.jpg',
     'name': "Men's Black Cargo Pants with Large Pockets",
     'description': "These men's black cargo pants are designed with a relaxed fit and feature conspicuously large, functional cargo pockets on the sides, emphasizing a utilitaria